<a href="https://colab.research.google.com/github/elsioantunes/proc-video/blob/main/procVideoLab7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Face detection 
É o processo pelo qual um sistema classifica e/ou filtra traços e formas para extrair a identificação de um rosto humano. Face detection é um tipo específico de detecção de objeto. Uma tecnologia (associada a matéria de visão computacional) que trata da identificação e significação através de imagens e videos.

A tecnologia de face detection se desdobra em outras utilidades além de simples identificação de uma forma humana. Como se trata de treinamento de redes neurais, o nível de detalhamento de um objeto identificável pode chegar a nuances inimagináveis. Um rosto pode ser identificado, uma pessoa específica pode ser identificada, estando ela com maquiagem, barba, óculos, máscara. Um animal pode ser identificado: o Google Photos registra suas fotos e identifica as pessoas e OS PETs das pessoas. 

Além da biometria e marketing, detecção facial pode chegar a niveis bem específicos de leitura labial e reconstrução facial! Atualmente (2021) tem surgido nas plataformas e redes socials de videos as chamadas *deepfakes* que utilizam treinamento extenso de rostos específicos para montagens (quase perfeitas) de rostos enxertados em modelos e/ou trechos de filmes com atores trocados.

## Objetivos:

1. Aplicar uma quantidade de conceitos de processamento de vídeo da disciplina.
 $\color{#DDDD00}{\text{em processo}}$
2.  Executar a aplicação com cenas pré-gravadas, para testes e gravação de “demo”. $\color{#DDDD00}{\text{em processo}}$
3.  Estudar e aplicar os mecanismos de face detection $\color{#00dd00}{\text{ok}}$


A implementação de uma aplicação de deteção de rostos envolve o treinamento de um banco de dados de imagens de diversos rostos de diversas pessoas, de diversas aparências em diversas posições e iluminações. Felizmente a biblioteca `openCV` já tem um banco de dados treinado e uma função que recebe uma imagem e retorna a posição da imagem onde deve haver um rosto (ou vários outros objetos detectáveis). 

Neste relatório, portanto, vamos nos limitar a implementação do argabouço necessário para transformar o fluxo obtido da câmera do dispositivo em um fluxo dodato de detecção facial.

Para isso, em python e especialmente, utilizando a plataforma Colab do google, precisaremos de uma interação entre Javascript e Python, dado o acesso ao dispositivo por uma plataforma online necessita de permissões dadas ao browser. 

In [ ]:
import numpy as np
import random as rnd
import cv2 as cv
from base64 import b64encode, b64decode
from google.colab.patches import cv2_imshow
from google.colab.output import eval_js
from IPython.display import HTML, Image, display, Javascript
 
def show(img):
    return cv2_imshow(img)
 
def rndColor():
    r = rnd.randrange(50, 240)
    g = rnd.randrange(50, 240)
    b = rnd.randrange(50, 240)
    return (r, g, b)
 
def dim3(img):
    if len(img.shape) == 2:
        img = cv.merge([img, img, img])
    return img.astype('uint8')
 
def newVideo(outUrl, frame, codec='VP80'):
    fourcc = cv.VideoWriter_fourcc(*codec)
    vheight, vwidth = frame.shape[:2]
    return cv.VideoWriter(outUrl, fourcc, 25, (vwidth, vheight))

Para TESTAR o código, sem o uso de câmera, estou utilizando um video pré gravado que pode ser baixado da seguinte forma. Mas o código final será feito diretamente com a câmera do dispositivo

In [ ]:
!curl -o v.mp4 https://elsioantunes.github.io/proc-video/faceDetecTest.mp4
!cp v.mp4 /usr/local/share/jupyter/nbextensions/v.mp4

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1598k  100 1598k    0     0  18.3M      0 --:--:-- --:--:-- --:--:-- 18.3M


Para executar usamos um elemento video em um conteúdo html. Como ele está nesta pasta específica, pode ser acessado pela tag `video`.
(mas não pela tag `img`, não sei qual a diferença)

In [ ]:
%%html
<video controls>
    <source src='nbextensions/v.mp4'>
</video>

## Detecção em video

Inicialmente, obtém-se os arquivos dos valores da rede previamente treinada. Neste caso, utilizamos os arquivos disponiveis no github oficial do algoritmo 
* https://github.com/opencv/opencv/tree/master/data/haarcascades

Lá você encontra os arquivos xml contendo o treinamento de redes especializadas em rostos, olhos, sorriso, gatos e placas de carro.

Em seguida, dentro de um loop de captura e gravação de video em python, já explicado nos relatórios anteriores, para cada frame obtido, após a dessatoração das cores aplicamos a função `cv.CascadeClassifier()` (definido em cada item nas primeiras linhas), especificamente em seu método `cv.CascadeClassifier().detectMultiScale()` com os parâmetros apropriados para a obtenção das coordenadas que definem o retângulo onde o algoritmo diz conter o ítem procurado.

In [ ]:
face = cv.CascadeClassifier(cv.samples.findFile(cv.data.haarcascades + 'haarcascade_frontalface_default.xml'))
olho1 = cv.CascadeClassifier(cv.samples.findFile(cv.data.haarcascades + 'haarcascade_righteye_2splits.xml'))
olho2 = cv.CascadeClassifier(cv.samples.findFile(cv.data.haarcascades + 'haarcascade_lefteye_2splits.xml'))
face2 = cv.CascadeClassifier(cv.samples.findFile(cv.data.haarcascades + 'haarcascade_frontalface_alt2.xml'))
sorriso = cv.CascadeClassifier(cv.samples.findFile(cv.data.haarcascades + 'haarcascade_smile.xml'))
perfil = cv.CascadeClassifier(cv.samples.findFile(cv.data.haarcascades + 'haarcascade_profileface.xml'))

inUrl = '/usr/local/share/jupyter/nbextensions/v.mp4'
outUrl = '/usr/local/share/jupyter/nbextensions/video.mp4'

def openVideo(url):
    cap = cv.VideoCapture(url)
    ret, frame = cap.read()
    h, w = frame.shape[:2]
    fps = cap.get(cv.CAP_PROP_FPS)
    fc = cap.get(cv.CAP_PROP_FRAME_COUNT)
    print("frame count:", fc)
    print("width, height, fps:", w, h, fps)
    return fc, w, h, cap

def newVideo(frame, outUrl='video.mp4', codec='VP90'):
    fourcc = cv.VideoWriter_fourcc(*codec)
    h, w = frame.shape[:2]
    return cv.VideoWriter(outUrl, fourcc, 25, (w, h))

out = None
ret = True
fc, w, h, cap = openVideo(inUrl)

for i in range(int(fc)):
#for i in range(15):
    data = []
    ret, frame = cap.read()
    if ret:
        gray = cv.cvtColor(frame, cv.COLOR_RGB2GRAY)

        olhos = olho1.detectMultiScale(gray, minNeighbors=15)
        for (x, y, w, h) in olhos:
            data.append({"target":"olho dir", "x":x, "y":y, "w":w, "h":h, "cor":(0, 0, 64)})

        olhos = olho2.detectMultiScale(gray, minNeighbors=15)
        for (x, y, w, h) in olhos:
            data.append({"target":"olho dir", "x":x, "y":y, "w":w, "h":h, "cor":(0, 64, 0)})

        perfis = perfil.detectMultiScale(gray)
        for (x, y, w, h) in perfis:
            data.append({"target":"perfil", "x":x, "y":y, "w":w, "h":h, "cor":(0, 128, 0)})

        faces = face2.detectMultiScale(gray, minNeighbors=15)
        for (x, y, w, h) in faces:
            data.append({"target":"face2", "x":x, "y":y, "w":w, "h":h, "cor":(128, 0, 0)})

            
        sorrisos = sorriso.detectMultiScale(gray, minNeighbors=125)
        for (x, y, w, h) in sorrisos:
            data.append({"target":"face2", "x":x, "y":y, "w":w, "h":h, "cor":(255, 255, 0)})

            
        for p in data:
            if p['target'].split()[0] == 'olho':
                cv.circle(frame, (p['x']+p['w']//2, p['y']+p['h']//2), p['w']//2, p['cor'], 2)
            else:
                cv.circle(frame, (p['x']+p['w']//2, p['y']+p['h']//2), p['w']//2, p['cor'], 1)
                cv.rectangle(frame, (p['x'], p['y']), (p['w']+p['x'], p['h']+p['y']), p['cor'], 1)

        
        if out == None: out = newVideo(frame, outUrl)
        
        out.write(frame)
        print (i, end=' ')
        if i % 30 == 0: print()
        '''
        show(frame)
        '''

cap.release()
out.release()

Após a obtenção dos retângulos em uma lista, que pode conter outras informações a respeito da captura, como por ex, o nome do que foi capturado, procede-se a marcação em uma imagem (preferencialmente o próprio frame corrente) com a ajuda dos comandos `cv.circle` e `cv.rectangle` com cores a gosto. 
* Neste trecho também seria interessante um refinamento. Dado que aqui se recebe informações sobre objetos vindos de treinamentos distintos, cálculos matemáticos e abordagens de coerência e simetria do rosto podem ser aplicados.

O próprio método `detectMultiScale()` já fornece opções em seus parâmetros para refinar a busca. Por exemplo, o parâmetro `minNeighbors` que compara os resultados de uma espécie de votação, como ocorre no algoritmo KNN, também já abordado em relatórios anteriores. O parâmetro é especialmente útil nos casos de captura de sorriso, dado que o padrão procurado também pode ser encontrado em diversas situações aleatórias da imagem. 





## Detecção em tempo real (com a câmera do celular)

* Inicialmente, vamos precisar acessar a câmera do dispositivo. No meu caso, eu não tenho webcam instalada no PC (e não vou comprar uma agora só para concluir o trabalho) mas posso usar a câmera do celular. Isso se faz em javascript utilizando o comando `navigator.mediaDevices.getUserMedia()` que por ser asyncrono deve ser instanciado adequadamente utilizando as tecnologias atuais de `async` e `await` em javascript nos navegadores modernos. 

* Para exibir o conteúdo da câmera é necessário a instalação de um elemento `video` que será instanciado em um código html gerado dinamicamente pelo javascript. 

* Além da tag `video`, também será instanciado dentro do mesmo `div` um `button` de disparo, um `canvas` para exibir em cima do conteúdo de video, alguns desenhos acionados por script e um pequeno campo de texto para exibição de mensagens (debug).

* A função `loop` fica rodando indefinidamente, atribuindo a variável `result` o conteúdo do instante exibido em video e capturado por canvas. Ali eu aproveito também para rabiscar alguma coisa sobre o conteúdo. Graças a função `requestAnimationFrame` que funciona assincronamente, executando o *callback* que é a própria função loop sendo colocada constantemente numa pilha de execução esperando a *thread* principal ser desocupada. 

* Por último, a função `streamFrame` fica responsável por retornar um frame do dispositivo para o sistema. Do lado Python eu posso rodar um loop que requisita em tempo real cada frame do dispositivo, tentando dentro do possível, sincronizar com o loop interno do lado javascript. A comunicação não é exatamente instantânea, mas é suficiente para que coisas como detecção de rostos seja bem sucedida. 



In [ ]:
def initJS():
    js = Javascript('''
        var stream
        var done = false
        var div = null
        var markData = [0]
        var marks = []
        var debug = null
        var clock = 0
        var video = null
        var canvas = null
        var result = null
        var pi = Math.PI
        var key = 0
 
        function createElement(el, pai){
            const handler = document.createElement(el)
            pai.appendChild(handler)
            return handler
        }
 
        function createVid(stream){
            const video = createElement('video', div)
            video.style.display = 'none'
            if (stream)
                video.srcObject = stream
            else    
                video.src = 'nbextensions/v.mp4'
            return video
        }
 
        function createBt(){
            const button = createElement('button', div)
            button.innerText = 'OK'
            button.onclick = () => {done = true}
            return button
        }
 
        function createCanvas(){
            canvas = createElement('canvas', div)
            canvas.style.display = 'block'
            canvas.width = video.width
            canvas.height = video.height
            canvas.onclick = () => {done = true}
            cnv = canvas.getContext('2d')
            return cnv
        }
 
        function kill(){
            if (stream)
                stream.getVideoTracks()[0].stop();
            canvas.remove()
            video.remove()
            button.remove()
        }
 
        function arco(x, y, a, b, r, cor){
            cnv.strokeStyle = cor
            cnv.beginPath()
            cnv.arc(x, y, r, -a, -b, true)
            cnv.stroke()
        }
 
        function clone(obj) {
            if (null == obj || "object" != typeof obj) return obj;
            copy = {};
            for (var attr in obj) {
                if (obj.hasOwnProperty(attr)) copy[attr] = clone(obj[attr]);
            }
            return copy;
        }
 
        function updateCanvas(){
            cnv.drawImage(video, 0, 0, video.width, video.height) 
            
            for (mark of marks){
                cnv.strokeStyle = mark.cor
                cnv.strokeRect(mark.x, mark.y, mark.w, mark.h)
            }
            
            if (markData[0] != key){
                key = markData[0]
                marks = []
                for (var i=1; i < markData.length; i++){
                    marks.push(clone(markData[i]))
                }
            }
        }
 
        function loop(){
            if (!done){
                clock++
                result = canvas.toDataURL('image/jpeg', 0.8)
                requestAnimationFrame(loop);
                updateCanvas()
            }
        }
 
        async function streamFrame(data){
            if (done){
                debug.innerText = "bye streamFrame!"
                kill()
                return [false, result, clock]
            }
            if (div == null){
                stream = await navigator.mediaDevices.getUserMedia({video: true})
                div = createElement('div', document.body)
                button = createBt()
                video = createVid(stream)
                await video.play()
                //video.width = 540
                //video.height = 360
                video.width = 360
                video.height = 360
 
                cnv = createCanvas()
                debug = createElement('div', div)
                cnv.strokeStyle = "red"
                loop()
            }
            markData = data
            return [true, result, clock]
        }
 
    ''')
    display(js)

Do lado Python da força, podemos acessar qualquer função predefinida em javascript através do comando `eval_js` que é a nossa ponte entre os dois mundos.

Depois de decodificar o jpeg vindo do canvas, aplico a cada frame o comando openCV responsável pela detecção de rosto. 

In [ ]:
face = cv.CascadeClassifier(cv.samples.findFile(cv.data.haarcascades + 'haarcascade_frontalface_default.xml'))
olho1 = cv.CascadeClassifier(cv.samples.findFile(cv.data.haarcascades + 'haarcascade_righteye_2splits.xml'))
olho2 = cv.CascadeClassifier(cv.samples.findFile(cv.data.haarcascades + 'haarcascade_lefteye_2splits.xml'))
 
face2 = cv.CascadeClassifier(cv.samples.findFile(cv.data.haarcascades + 'haarcascade_frontalface_alt2.xml'))
sorriso = cv.CascadeClassifier(cv.samples.findFile(cv.data.haarcascades + 'haarcascade_smile.xml'))
perfil = cv.CascadeClassifier(cv.samples.findFile(cv.data.haarcascades + 'haarcascade_profileface.xml'))
 
def streamFrame(arg):
    return eval_js(f'streamFrame({arg})')
 
'''
'MessageError: NotSupportedError: Failed to load because no supported source was found'
    significa que, no modo video, o mp4 não foi carregado. 
    execute a celula de download, ou troque por um link
'''
 
def decodeCanvasData(data):
    binary = b64decode(data.split(',')[1])
    binary = np.frombuffer(binary, 'uint8')
    img = cv.imdecode(binary, 1)
    #cv2_imshow(img)
    return img
 
initJS()
ret = True
frame = None
data = []
 
while ret:
    ret, buf, clock = streamFrame(data)
    data = [clock]
    if ret:
        frame = decodeCanvasData(buf)
        gray = cv.cvtColor(frame, cv.COLOR_RGB2GRAY)
        
        '''
        sorrisos = sorriso.detectMultiScale(gray)
        for (x, y, w, h) in sorrisos:
            data.append({"target":"sorriso", "x":x, "y":y, "w":w, "h":h, "cor":"olive"})
        '''
        
        olhos = olho2.detectMultiScale(gray)
        for (x, y, w, h) in olhos:
            data.append({"target":"olho esq", "x":x, "y":y, "w":w, "h":h, "cor":"blue"})
        
        olhos = olho1.detectMultiScale(gray)
        for (x, y, w, h) in olhos:
            data.append({"target":"olho dir", "x":x, "y":y, "w":w, "h":h, "cor":"black"})
        
        faces = face.detectMultiScale(gray)
        for (x, y, w, h) in faces:
            data.append({"target":"face", "x":x, "y":y, "w":w, "h":h, "cor":"green"})
        
        faces = face2.detectMultiScale(gray)
        for (x, y, w, h) in faces:
            data.append({"target":"face2", "x":x, "y":y, "w":w, "h":h, "cor":"lime"})
        
        faces = perfil.detectMultiScale(gray)
        for (x, y, w, h) in faces:
            data.append({"target":"face2", "x":x, "y":y, "w":w, "h":h, "cor":"gray"})
        
show(frame)

<IPython.core.display.Javascript object>

# REFERÊNCIAS



* Classificadores treinados para captura de objetos <br> 
https://github.com/opencv/opencv/tree/master/data

* Tutorial Cascade Classifier OpenCV <br> 
https://docs.opencv.org/4.5.3/db/d28/tutorial_cascade_classifier.html

* Object Detection using Haar feature-based cascade classifiers <br> 
<a href='https://colab.research.google.com/github/computationalcore/introduction-to-opencv/blob/master/notebooks/4-Cascade_classification.ipynb#scrollTo=T2JCR1Z47g9L'>https://colab.research.google.com/github/computationalcore/</a>

* Motion Detection OpenCV Python <br> 
https://divyanshushekhar.com/motion-detection-opencv/

* Motion Analysis <br> 
https://docs.opencv.org/4.5.2/de/de1/group__video__motion.html





